In [1]:
import sys
sys.path.append('/home/wangs/rs/lib')
import ff
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
sns.set_palette("pastel")
from multiprocessing import Pool
from tqdm import tqdm
import statsmodels.api as sm
from datetime import datetime
import mplfinance as mpf
from functools import partial

In [2]:
def daily_K_line(event,n,open,close,high,low):
    # 日均事件K线
    T0open=event*open  # 以T0open作为基准
    pre_close = event * close.shift(axis = 1)
    data=pd.DataFrame(columns=['T'+str(j) for j in range(n)])
    for j in range(n):
        _open=(event*open.shift(-j,axis=1))/pre_close
        _close=(event*close.shift(-j,axis=1))/pre_close
        _low=(event*low.shift(-j,axis=1))/pre_close
        _high=(event*high.shift(-j,axis=1))/pre_close
        lst=[_open.mean().mean(),_close.mean().mean(),_low.mean().mean(),_high.mean().mean()]
        data['T'+str(j)]=lst
    data.index=['open','close','low','high']
    data=data.T
    data.index=pd.to_datetime(['20200101','20200102','20200103','20200104','20200105'])
    # mpf.plot(data,type='candle',style='default')
    return data

In [3]:
stock_index = ff.read('一字板').index
time_index = ff.read('一字板').columns
up = ff.read('up').loc[stock_index,time_index].copy()
# open = ff.read('open').loc[stock_index,time_index].copy()
close = ff.read('close').loc[stock_index,time_index].copy()
high = ff.read('high').loc[stock_index,time_index].copy()
low = ff.read('low').loc[stock_index,time_index].copy()
pre_close = ff.read('pre_close').loc[stock_index,time_index].copy()
up_limit = pd.DataFrame(ff.read('up_limit').copy(),index = stock_index,columns = time_index) * ff.filter0.loc[stock_index,time_index]

In [4]:
up_filter = (high == up_limit)
dict = {}
for i in up_filter.sum(axis = 1)[up_filter.sum(axis = 1)>0].index:
    stock = up_filter.loc[i,:]
    dict[i] = list(stock[stock == True].index)

In [5]:
def up_first(stock):
    first = []
    for time in dict[stock]:
        start_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 09:31:00'
        end_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 15:00:00'
        data = ff.read_min(stock).loc[start_time:end_time,:]
        daily_up_limit = data.loc[:,'high_limit']
        start_1 = np.argwhere((data.loc[:,'high'] == daily_up_limit).values)[0][0]
        first.append(start_1)
    return first
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(up_first,dict.keys()),total = len(dict.keys())))
up_first_factor = pd.DataFrame(np.nan,index = up.index,columns = up.columns)
for i in range(len(res_lst)):
    for j in range(len(res_lst[i])):
        up_first_factor.loc[list(dict.keys())[i],list(dict.values())[i][j]] = res_lst[i][j]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4414/4414 [03:56<00:00, 18.67it/s]


In [6]:
factor_names1=['closer', 'openr', 'lowr', 'highr', 'ocret', 'lcret', 'hcret', 'ohret', 'ret5', 'ret10', 'ret20', 'ret30', 'excess5', 'excess10', 'excess20', 'excess30', 'ma5', 'bias5', 'ma10',
               'bias10', 'ma15', 'bias15', 'ma20', 'bias20', 'vwap_ret', 'vwaph_ret', 'll5', 'll15', 'll20', 'll30', 'a0', 'a5', 'a15', 'a30', 'ah5', 'ah10', 'ah15', 'std', 'amt_std', 'vol_std', 
               'amt_mean', 'vol_mean', 'turnover', 'turnover_mean', 'turnover_std', 'turnover_skew', 'turnoverf', 'turnoverf_mean', 'turnoverf_std', 'turnoverf_skew', 'mkv', 'l_mkv', 'lhret', 
               'STR', 'MTR', 'SMTR', 'ILLIQ','close','high','low']
factor_names2=['ARRP', 'ARRP_5d_20mean', 'AmpMod', 'BVol', 'CDPDVP', 'ILLIQ', 'IVoldeCorr', 'IntraDayMom20', 'MTR', 'RCP', 'SMTR', 'SPS', 'STR', 'a30', 
               'afternoon_ratio_1', 'ah15', 'amt', 'amt_ma20', 'amt_mean', 'amt_score0', 'amt_score1', 'amt_score2', 'amt_std', 'amt_std20', 'bias10', 'bias15', 'bias20', 'bias5', 
               'cross_std10', 'cross_std10adj', 'ctrade', 'excess10', 'excess20', 'excess30', 'excess5', 'growth_a', 'growth_b', 'growth_c', 'h_rankamt_corr_10', 'h_rankv_corr_10', 'h_rankv_corr_36', 
               'hcret', 'idiov', 'intro_high80_corr', 'ivr', 'l_mkv', 'mkv', 'mom1', 'mom_spring', 'nl_mkv', 'nl_mom', 'open0931ratio', 'open5ratio_ma10vol', 'rank_2_corr_hv10', 'rank_2_corr_hv20', 'rank_ha_corr_sum', 
               'ret20', 'ret30', 'ret5', 'roc121', 'roc20', 'roc240', 'roc5', 'scr', 'std', 'std20', 'std21', 'std240', 'stddev_cov', 'stddev_cov_right', 'sub', 'sysv', 'tail_ratio_1', 'tailrets0', 'tailrets1', 'tliq', 
               'turn20', 'turnover', 'turnover_mean', 'turnover_scale_z', 'turnover_score_ts_mean', 'turnover_score_ts_std', 'turnover_skew', 'turnover_std', 'turnoverf', 'turnoverf_mean', 'turnoverf_skew', 'turnoverf_std', 
               'up_KURS', 'up_plus_down_KURS', 'vol', 'vol_mean', 'vol_std', 'vol_std20', 'volroc_skew', '滞后日内量价corr']
factor_name3 =['accelerated_turnover_rank_RC','CSK_XYY_UP_DOWN_120D_RC','high_fre_vol_RC','high_fre_diff_vol_RC','high_fre_absdiff_vol_RC','peak_count_vol_RC','overnightsmart20_RC','CTR_RC','jumpCTR_RC','turnover_rate_proportion_l','synergy']
factor_name4 = ['ll60','ll120', 'lr5', 'lr10', 'lr20', 'lr30','lr60','lr120','posi60','posi120','posi240','nhigh20','nhigh60', 'nhigh120','nhigh20','nhigh60','nhigh120']
factor_name5 = ['rtn_condVaR', 'CTR_RC', 'jumpCTR_RC', 'openr', 'CCOIV', 'lr30', 'SZXZ', 'vol_DW', 'buy_sm_amount', 'lr20', 'TCV', 'ah10', 'buy_elg_amount', 'ret10', 'vol_LBQ', 'ah5', 'lr120', 'ma15', 'correlation_matrix13', 'high_fre_diff_vol', 'posi120', 'nlow60', 'roc30', 'accelerated_turnover_rank', 'VolStd', 'low', 'sell_sm_amount', 'vwap_ret', 'rtn_DW', 'high', 'sell_md_amount', 'll5', 'SBZL', 'lhret', 'nhigh120', 'SQ', 'draw30', 'lr60', 'SPR', 'close', 'rtn_LBQ', 'WBGM', 'closer', 'SCOV', 'll60', 'feph', 'ID_Vol_deCorr', 'highr', 'NCV', 'buy_md_amount', 'lcret', 'SCCOIV', 'VoPC', 'highStdRtn_meanN', 'lowr', 'up', 'nhigh20', 'll20', 'high_fre_vol', 'ma5', 'ZMCW', 'bias30', 'high_fre_absdiff_vol', 'a5', 'nlow120', 'UID', 'll15', 'sell_elg_amount', 'posi60', 'a0', 'ma20', 'roc60', 'correlation_matrix12', 'overnightsmart20_RC', 'peak_count_vol', 'roc15', 'lr10', 'VoWVR', 'rtn_rho', 'nhigh60', 'down', 'posi240', 'foc_Comb', 'vwaph_ret', 'vol_rho', 'GYCQ', 'draw60', 'up_limit', 'post', 'PCV', 'UTD10', 'RPV', 'buy_lg_amount', 'draw15', 'ocret', 'ma10', 'll30', 'rtn_foc', 'sell_lg_amount', 'vwap', 'YMSL', 'ohret', 'net_mf_amount', 'DW_Comb', 'market_mean_IV', 'CSK_XYY_UP_DOWN_120D', 'correlation_matrix11', 'down_limit', 'HYLJ', 'nlow20', 'bias60', 'SRV', 'LBQ_Comb', 'UTD20', 'open', 'a15', 'COYCYV', 'rho_Comb', 'll120', 'highStdRtn_mean', 'lr5', 'vol_foc']
Gpalpha_factor = ['Gpalpha001','Gpalpha002','Gpalpha003','Gpalpha004','Gpalpha005','Gpalpha006','Gpalpha007','Gpalpha008','Gpalpha009','Gpalpha010','Gpalpha011','Gpalpha012','Gpalpha013','Gpalpha014','Gpalpha015']
fields = factor_names1 + factor_names2 + factor_name3 + factor_name4 + factor_name5
# fields = Gpalpha_factor
names = list(set(fields))

In [7]:
def five_quantile(x):
    result = x.copy()
    x[(result<=result.quantile(0.2))] = 0
    x[(result>result.quantile(0.2))*(result<=result.quantile(0.4))] = 1
    x[(result>result.quantile(0.4))*(result<=result.quantile(0.6))] = 2
    x[(result>result.quantile(0.6))*(result<=result.quantile(0.8))] = 3
    x[(result>result.quantile(0.8))] = 4
    return x

In [8]:
shape_data =  (ff.read('单次涨停').copy().replace({np.nan:0}) + ff.read('多次涨停').copy().replace({np.nan:0}) + ff.read('炸板股').copy().replace({np.nan:0})).replace({0:np.nan}) * (ff.read('10cm').copy())
feature_data = ((up_first_factor.copy() < 1) | (up_first_factor.copy() >= 180)).replace({False:np.nan,True:1.0})* (ff.read('10cm').copy())

In [9]:
def factor_touch(factor_name):
    stat = pd.DataFrame(columns = ['因子','因子分组','mean','median','ratio','T0日内收益','T1开盘收益','T1日内收益','合计收益'])
    factor = pd.DataFrame(ff.read(factor_name),index = stock_index,columns=time_index).copy().apply(five_quantile).shift(axis = 1)
    for j in range(5):
        factor_ = (factor == j).replace({False:np.nan,True:1.0})
        event_ =  shape_data * feature_data * factor_
        data = daily_K_line(event_,5,open_,close_,high_,low_)
        stat.loc[len(stat.index)] = [factor_name,j,event_.sum().mean(),event_.sum().median(),(event_.sum()>0).sum()/event_.shape[1],(data.iloc[0,1] -1),(data.iloc[1,0] - data.iloc[0,1]),(data.iloc[1,1] - data.iloc[0,1]),(data.iloc[0,1] -1-0.1+data.iloc[1,0] - data.iloc[0,1])]
    return stat

In [10]:
open_ = (ff.read('open') * ff.read('post')).loc[stock_index,time_index].copy()
close_ = (ff.read('close')* ff.read('post')).loc[stock_index,time_index].copy()
high_ = (ff.read('high')* ff.read('post')).loc[stock_index,time_index].copy()
low_ = (ff.read('low')* ff.read('post')).loc[stock_index,time_index].copy()

In [11]:
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(factor_touch,names),total = len(names)))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 226/226 [05:05<00:00,  1.35s/it]


In [12]:
result = pd.concat(res_lst)

In [13]:
result = result.sort_values(by = '合计收益',ascending = True)
result.head(20)

,因子,因子分组,mean,median,ratio,T0日内收益,T1开盘收益,T1日内收益,合计收益
3,ll5,3,0.539290,0.0,0.264801,0.058493,-0.013446,-0.020309,-0.054953
4,ll5,4,1.469322,1.0,0.686760,0.061457,-0.008143,-0.009235,-0.046686
2,ll20,2,0.023681,0.0,0.007535,0.052595,0.001409,0.001242,-0.045996
4,lr5,4,2.406889,2.0,0.784715,0.064734,-0.006314,-0.006613,-0.041580
0,up,0,3.044133,3.0,0.829925,0.061425,-0.001503,-0.006152,-0.040078
0,lhret,0,1.079656,1.0,0.582347,0.066907,-0.005481,-0.005906,-0.038574
2,lr20,2,0.026911,0.0,0.004306,0.062141,-0.000061,0.029500,-0.037920
0,buy_sm_amount,0,1.372443,1.0,0.655544,0.063817,-0.001447,-0.004781,-0.037630
4,lr10,4,2.852530,2.0,0.814855,0.068519,-0.004862,-0.005209,-0.036343
4,ll15,4,1.951561,1.0,0.742734,0.070987,-0.007059,-0.007097,-0.036071


因子值减去当天的中位数

In [ ]:
def factor_touch_med(factor_name):
    stat = pd.DataFrame(columns = ['因子','因子分组','mean','median','ratio','T0日内收益','T1开盘收益','T1日内收益','合计收益'])
    factor = pd.DataFrame(ff.read(factor_name) - ff.read(factor_name).median(),index = stock_index,columns=time_index).copy()
    event = (factor * shape_data * feature_data).copy().apply(five_quantile).shift(axis = 1)
    for j in range(5):
        event_ = (event == j).replace({False:np.nan,True:1.0})
        data = daily_K_line(event_,5,open_,close_,high_,low_)
        stat.loc[len(stat.index)] = [factor_name,j,event_.sum().mean(),event_.sum().median(),(event_.sum()>0).sum()/event_.shape[1],(data.iloc[0,1] -1),(data.iloc[1,0] - data.iloc[0,1]),(data.iloc[1,1] - data.iloc[0,1]),(data.iloc[0,1] -1-0.1+data.iloc[1,0] - data.iloc[0,1])]
    return stat
with Pool(24) as p:
    res_lst2 = list(tqdm(p.imap(factor_touch_med,names),total = len(names)))
result2 = pd.concat(res_lst2)

 13%|████████████████▎                                                                                                              | 29/226 [01:05<06:45,  2.06s/it]

In [ ]:
result2 = result2.sort_values(by = '合计收益',ascending = True)
result2

In [35]:
import pickle 
with open('/home/wangs/data/feature/因子_首次触板时间分组.pk','wb') as f:
    pickle.dump(result,f)

前一天涨停一字板 + 第二天触板时刻 + 第二天开盘未涨停

In [31]:
stat_up = pd.DataFrame(columns = ['首次触板时间分组','mean','median','ratio','T0日内收益','T1开盘收益','T1日内收益','合计收益'])
shape_data =  ff.read('一字板').loc[stock_index,time_index].copy().shift(axis = 1)
feature_data = (up_first_factor.copy().applymap(setg)) * (ff.read('10cm').copy())
open_up = (ff.read('open').loc[stock_index,time_index].copy() == up_limit).replace({False:1.0,True:np.nan})
for i in range(0,2):
    num_ = (feature_data == i).replace({False:np.nan,True:1.0})
    event_ =  shape_data * num_ * open_up 
    data = daily_K_line(event_,5,open_,close_,high_,low_)
    stat_up.loc[len(stat_up.index)] = [i,event_.sum().mean(),event_.sum().median(),(event_.sum()>0).sum()/event_.shape[1],(data.iloc[0,1] -1),(data.iloc[1,0] - data.iloc[0,1]),(data.iloc[1,1] - data.iloc[0,1]),(data.iloc[0,1] -1-0.1+data.iloc[1,0] - data.iloc[0,1])]

In [32]:
stat_up.sort_values(by = '合计收益',ascending = False)

,首次触板时间分组,mean,median,ratio,T0日内收益,T1开盘收益,T1日内收益,合计收益
1,1.0,1.247578,1.0,0.625404,0.028265,-0.012571,-0.003507,-0.084306
0,0.0,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN


前一天涨停不包括一字板 + 第二天触板时刻

In [51]:
stat = pd.DataFrame(columns = ['首次触板时间分组','mean','median','ratio','T0日内收益','T1开盘收益','T1日内收益','合计收益'])
shape_data =  (ff.read('一字板').loc[stock_index,time_index].copy().replace({np.nan:1,1.0:np.nan}) * ff.read('up').copy()).shift(axis = 1)
feature_data = (up_first_factor.copy().applymap(setg)) * (ff.read('10cm').copy())
for i in range(0,9):
    num_ = (feature_data == i).replace({False:np.nan,True:1.0})
    event_ =  shape_data * num_ 
    data = daily_K_line(event_,5,open_,close_,high_,low_)
    stat.loc[len(stat.index)] = [i,event_.sum().mean(),event_.sum().median(),(event_.sum()>0).sum()/event_.shape[1],(data.iloc[0,1] -1),(data.iloc[1,0] - data.iloc[0,1]),(data.iloc[1,1] - data.iloc[0,1]),(data.iloc[0,1] -1-0.1+data.iloc[1,0] - data.iloc[0,1])]

In [52]:
stat.sort_values(by = '合计收益',ascending = False)

,首次触板时间分组,mean,median,ratio,T0日内收益,T1开盘收益,T1日内收益,合计收益
0,0.0,1.086068,0.0,0.241486,0.075495,0.026497,0.019089,0.001992
3,3.0,0.417337,0.0,0.195046,0.076012,0.011002,0.011631,-0.012986
1,1.0,0.216099,0.0,0.143344,0.070603,0.015755,0.012213,-0.013642
6,6.0,0.256966,0.0,0.146130,0.081271,0.002559,0.010113,-0.016170
7,7.0,0.229102,0.0,0.135294,0.082502,0.000517,0.007051,-0.016980
2,2.0,0.289474,0.0,0.166873,0.070333,0.011433,0.008037,-0.018234
4,4.0,0.289783,0.0,0.157895,0.073525,0.006609,0.007533,-0.019866
5,5.0,0.275851,0.0,0.157895,0.076113,0.003082,0.007864,-0.020806
8,8.0,0.188854,0.0,0.118885,0.087334,-0.009664,-0.010449,-0.022331


因子 + 当天的开盘涨幅 + 首次触板时刻 + 前一天涨停不包括一字板

In [71]:
open_ret = ff.read('open').loc[stock_index,time_index].copy() /pre_close
open_ret = open_ret.apply(five_quantile) #当天的开盘涨幅分组

In [72]:
shape_data =  (ff.read('一字板').loc[stock_index,time_index].copy().replace({np.nan:1,1.0:np.nan}) * ff.read('up').copy()).shift(axis = 1) #前一天涨停不包括一字板
feature_data = (up_first_factor.copy().applymap(setg)) * (ff.read('10cm').copy())

In [73]:
def factor_touch(factor_name):
    stat = pd.DataFrame(columns = ['因子','因子分组','首次触板时间分组','mean','median','ratio','T0日内收益','T1开盘收益','T1日内收益','合计收益'])
    factor = pd.DataFrame(ff.read(factor_name),index = stock_index,columns=time_index).copy().apply(five_quantile).shift(axis = 1)
    for j in range(5):
        factor_ = (factor == j).replace({False:np.nan,True:1.0})
        for i in range(0,9):
            num_ = (feature_data == i).replace({False:np.nan,True:1.0})
            event_ =  shape_data * num_ * factor_
            data = daily_K_line(event_,5,open_,close_,high_,low_)
            stat.loc[len(stat.index)] = [factor_name,j,i,event_.sum().mean(),event_.sum().median(),(event_.sum()>0).sum()/event_.shape[1],(data.iloc[0,1] -1),(data.iloc[1,0] - data.iloc[0,1]),(data.iloc[1,1] - data.iloc[0,1]),(data.iloc[0,1] -1-0.1+data.iloc[1,0] - data.iloc[0,1])]
    return stat
with Pool(24) as p:
    res_lst2 = list(tqdm(p.imap(factor_touch,names),total = len(names)))
result2 = pd.concat(res_lst2)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 123/123 [46:44<00:00, 22.80s/it]


In [74]:
for j in range(5):
    open_ret_ = (open_ret == j).replace({False:np.nan,True:1.0})
    for i in range(0,9):
        num_ = (feature_data == i).replace({False:np.nan,True:1.0})
        event_ =  shape_data * num_ * open_ret_
        data = daily_K_line(event_,5,open_,close_,high_,low_)
        result2.loc[len(result2.index)] = ['开盘涨跌幅',j,i,event_.sum().mean(),event_.sum().median(),(event_.sum()>0).sum()/event_.shape[1],(data.iloc[0,1] -1),(data.iloc[1,0] - data.iloc[0,1]),(data.iloc[1,1] - data.iloc[0,1]),(data.iloc[0,1] -1-0.1+data.iloc[1,0] - data.iloc[0,1])]